In [1]:
from joblib import delayed,Parallel
from nibabel import freesurfer as fs
import pandas as pd
import numpy as np
import os,glob
import re
import cPickle as pkl
from sklearn.feature_selection import variance_threshold
from sklearn.utils import resample
from joblib import Parallel,delayed
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection.univariate_selection import (
    check_X_y,safe_sparse_dot,issparse,row_norms,stats)
from pandas.core.algorithms import rank
import time
idx=pd.IndexSlice

In [2]:
#subject directory and subject list
gitpath=os.getcwd()
SUBJECTS_DIR=(
    '/Volumes/Users/mbkranz/projects/'
    'ACT_Freesurfer_NewProc/'
)
os.chdir(SUBJECTS_DIR)
sublist=!ls -d ACT*_1
os.chdir(gitpath)
idx=pd.IndexSlice

In [3]:
def compute_beta(X,Y):
    '''uses the equation (X'X)^(-1)X'Y 
    to calculate OLS coefficient estimates:
    equivalent to the first element 
    in the built in OLS fxn: np.linalg.lstsq
    X : features to predict Y
    Y : target variable(s)
    
    Returns an np.array of num vars
    of X by number of vars of Y
    '''
    n=len(X)
    X = np.column_stack([np.ones(n),np.array(X)])
    inv_dot_XX=np.linalg.inv(np.dot(X.T,X))
    dot_XY=np.dot(X.T,Y)
    betas=np.dot(inv_dot_XX,dot_XY)
    return betas

In [4]:
def compute_corr(X,y,spearman=True,center=True):
    '''
    this is taken from sklearn f_regression except option to rank
    variables and outputs correlations instead of f values
    '''
    if spearman==True:
        X=rank(X)
        y=rank(y)
    #allow y to be multi-dimensional
    X, y = check_X_y(X, y, ['csr', 'csc', 'coo'], 
                     dtype=np.float64)
    n_samples = X.shape[0]

    # compute centered values
    # note that E[(x - mean(x))*(y - mean(y))] = E[x*(y - mean(y))], 
    # so we need not center X
    if center:
        y = y - np.mean(y)
        if issparse(X):
            X_means = X.mean(axis=0).getA1()
        else:
            X_means = X.mean(axis=0)
        # compute the scaled standard deviations via moments
        X_norms = np.sqrt(row_norms(X.T, squared=True) -
                          n_samples * X_means ** 2)
    else:
        X_norms = row_norms(X.T)

    # compute the correlation
    corr = safe_sparse_dot(y, X)
    corr /= X_norms
    corr /= np.linalg.norm(y)
    #no pv needed as stats are done with bootstrap
    return corr

In [5]:
def compute_pcorr(X,y,covars):
    '''Extracts the residuals of covars on y
    and then computes the correlation 
    (also called a part correlation. 
    In other words, what is the relationship
    of X on Y after accounting for covars?
    X : 2-D np.array of observation X features
    Y : 1-D array of target variable
    covars : 1 or 2 D array of covariates to account for
    before running X~Y (residuals) correlation'''
    model=LinearRegression()
    model.fit(covars,y)
    resids=y-model.predict(covars)
    pcorr=compute_corr(X,resids)
    return pcorr

In [6]:
def run_mediate(mediatevar_mat,xvar_vec,covar_vec,yvar_vec):
    '''
    Note: make code to run with 1 or more mediator variables
    runs:
    1. direct effect or 
    yvar_vec=intercept+beta_c(xvar_vec) 
    e.g., Age~intercept+betaCognition
    2. full model
    run full model on each variable to get beta of xvar_vec
    when mediating variable is included
    (if more than one variable, will loop through)


    mediatevar_mat : a subject x Nvars numpy array of third
    variable (mediating variable)
    xvar_vec : subject-wise numpy vector of the first variable in 
    mediation of xvar_vec on yvar_vec
    yvar_vec : subject wise numpy vector of second variable of
    mediation of xvar_vec on yvar_vec
    covar_vec : subject-wise numpy vector of covariates to include
    (control for the effect of nuisance variables of no interest on 
    the relationship of xvar_vec on other variables)
    '''
    def compute_beta_withmediator():
        # try/except (for medial wall with all zero vals)
        X_mat=np.column_stack([xvar_vec,covar_vec,mediatevar_vec])
        try:
            beta_xvar=compute_beta(X_mat,yvar_vec)
        except:
            beta_xvar=np.repeat(np.nan,X_mat.shape[1])
        return beta_xvar[1]

    def compute_beta_withoutmediator():
        X_mat=np.column_stack([xvar_vec,covar_vec])
        beta_xvar=compute_beta(X_mat,yvar_vec)
        return beta_xvar[1]
    
    try:
        beta_c=compute_beta_withoutmediator()
        mediatevar_iter=np.nditer(mediatevar_mat,
                                  flags=['external_loop'],
                                  order='F')
        beta_cprime=np.array([compute_beta_withmediator()
                              for mediatevar_vec in mediatevar_mat.T])
        indirect=beta_c-beta_cprime
    except:
        indirect=np.nan
    return indirect

In [7]:
#make a boot class object to update
class RollingBootStats:
    '''Keeps all boot strap statistics when iterating 
    across bootstrapped samples.
    numsamples : total number of bootstrapped samples
    stat_labels: can provide an index to make 
    a pandas df with after analysis is done (or other labels to 
    identify the object from other Rolling BootStats objects)'''
    def __init__(self,stat_labels,numsamples=1):
        self.numsamples=numsamples
        self.stat_labels=stat_labels
        
    def update(self,bootobj,num_boot):
        self.num_boot=num_boot
        if num_boot==0: #whole sample
            self.empirical_mean=bootobj
        elif num_boot==1: #first boot sample, initiate objs
            self.bootobj_sum=bootobj
            self.bootobj_sum_diff2=0
        else: #compute rolling stats
            self.bootobj_sum=np.add(self.bootobj_sum,bootobj)
            bootobj_mean=self.bootobj_sum/num_boot
            self.bootobj_sum_diff2+=(
                bootobj - bootobj_mean)*(bootobj - bootobj_mean)
        if num_boot==self.numsamples:
            self.bootobj_mean=bootobj_mean
            self.bootobj_var=(self.bootobj_sum_diff2 / 
                              (num_boot - 1))
            self.bootobj_sd=np.sqrt(self.bootobj_var)
    def add_bootsample(self,bootobj,num_boot):
        if num_boot==1:
            self.replications=bootobj
        elif num_boot>1:
            self.replications=np.vstack([self.replications,bootobj])

In [8]:
def get_resampled_indices(npdata,numboot):
    '''resample with replacement the npdata
    from the csv file and extract the identifiers 
    across datasets (e.g., subs_resampled). These
    will be reused for the brain data indices.
    Make sure they are the same!!!!!'''
    if numboot==0:
        return npdata.index.values.copy(),npdata.copy()
    else:
        npdata_resampled=resample(npdata.copy())
        subs_resampled=npdata_resampled.index.values
        return subs_resampled,npdata_resampled

In [9]:
def load_format_cv_predict_df(cv_predict_path,threshtype=None):
    groupby_vars=['subs','measure','network','type','threshold','np_measure_name']
    df=pd.read_csv(cv_predict_path)
    df['type']=threshtype
    df['threshold']=np.round(df['threshold'],6).astype(str)
    df['network']=df['pkldir'].str.extract('(network\d)',expand=False)
    df['measure']=df['pkldir'].str.extract('(thickness|area)',expand=False)
    df['predictions']=np.where(df['predictions'].isnull(),0,df['predictions'])
    del df['pkldir']
    df_return=(df.filter(groupby_vars+['predictions'])
               .groupby(groupby_vars)
               .mean())
    return df_return['predictions']

In [10]:
def load_format_cv_predict_df_splits(cv_predict_path,threshtype=None):
    groupby_vars=['cviter', 'cvsplit','subs','testindex',
                 'measure','network','type','threshold','np_measure_name']
    df_withnans=pd.read_csv(cv_predict_path)
    nans=df_withnans.predictions.isnull()
    df=df_withnans.loc[~nans,:].copy()
    df['type']=threshtype
    df['threshold']=np.round(df['threshold'],6).astype(str)
    df['network']=df['pkldir'].str.extract('(network\d)',expand=False)
    df['measure']=df['pkldir'].str.extract('(thickness|area)',expand=False)
    del df['pkldir']
    return df

In [11]:
def init_stats_dict(measure,np_name,morph_df,numsamples=1):
    '''Initiates a RollingBootStats object after calculating the 
    number of columns (analyses) in the object.
    This is necessary as we need to split the dataset based on
    morphometry measures and cognitive measures before running analyses
    and each will a different number of columns 
    (as some thresholds etc were not included if they didn't
    #have any featuers selected in cv pipelinepredictions)'''
    morph_columns=morph_df.loc[:,idx[measure,:,:,:,np_name]].columns
    rollingstats=RollingBootStats(morph_columns,numsamples)
    return rollingstats

## Prediction analyses

In [ ]:
#read in npdata and morphometry predictions obtained from cv_results
#and processed in make_cv_dfs.R script
npdatafilt=pd.read_csv('../data/np_filter_wb_gendernum.csv',
                       na_values='NA',index_col="subs")

In [ ]:
groupby_vars=['subs','measure','network','type','threshold','np_measure_name']
cv_predict_df_fpr=load_format_cv_predict_df(
    '../data/cv_results/cv_predict_fpr_df_3fold_4_5_18.csv','fpr')
#filter out analyses that had no features selected on at least one fold 
to_include=((cv_predict_df_fpr==0).groupby(groupby_vars[1:]).sum()==0)
cv_predict_df_fpr=(
    cv_predict_df_fpr
    .reset_index(['subs'])
    .loc[to_include.loc[to_include==True].index.values]
    .reset_index()
    .set_index(groupby_vars)
    ['predictions'])

cv_predict_df_fpr_avg=(
    cv_predict_df_fpr
    .groupby(level=[x for x in groupby_vars if x!='threshold'])
    .mean()
    .reset_index()
    .assign(threshold='average')
    .set_index(groupby_vars)
    ['predictions'])

cv_predict_df_fpr_all=pd.concat([cv_predict_df_fpr,cv_predict_df_fpr_avg])

In [ ]:
cv_predict_df_fpr.head()

In [ ]:
cv_predict_df_percentile=load_format_cv_predict_df(
    '../data/cv_results/cv_predict_percentile_df_5fold_4_4_18.csv','percentile')
cv_predict_df_all=pd.concat((cv_predict_df_fpr_all,cv_predict_df_percentile))

In [ ]:
#unstack predictions (so rows are subjects) 
#unstacked columns need to be fully lexasorted for later slicing
cv_predict_df_unstack=(cv_predict_df_fpr_all
                       .unstack(groupby_vars[1:])
                       .sort_index(axis=1,level=[0,1,2,3,4]))
cv_predict_df_unstack.index=pd.Index(
    cv_predict_df_unstack.index.values.astype(int),name='subs')

In [ ]:
def make_statdf(name):
    stat_regex='^(area|thickness)_(Memory|ExFunction)_'
    if name=='lower':
        stat_str='np.percentile(stat.replications,axis=0,q=5)'
    elif name=='upper':
        stat_str='np.percentile(stat.replications,axis=0,q=95)'
    else:
        stat_str='stat.'+name
    print(stat_str)
    stat_df=pd.concat([(pd.DataFrame(eval(stat_str),index=stat.stat_labels)
                        .assign(analysis=re.sub(stat_regex,'',key),
                                stat=name))
                       for key,stat in bootstats_dict.iteritems()])
    return stat_df 

In [ ]:
numsamples=2000
num_boot=0
#number of analyses --> numcols in each sliced df 
#(slicing on np_name and cog)

bootstats_dict={meas+'_'+np_name+'_'+anal:init_stats_dict(
    meas,np_name,cv_predict_df_unstack,numsamples)
                for meas in measure_list 
                for np_name in np_name_list 
                for anal in anal_list}
for samplenum in range(numsamples+1):
    ###resampled npdata and morph data
    subs,npdata=get_resampled_indices(npdatafilt,num_boot)
    age=-npdata['Age'].values
    gender=npdata[['Gender']].values
    #loop through each cognitive measure and morph measure
    for np_name in np_name_list:
        cognitive=npdata[np_name].values
        for measure in measure_list:
            ###resampled whole brain averages 
            ###(specific to each measure)
            gender_wholebrain=npdata[
                ['Gender','_'.join(['wholebrain',measure])]].values
            #slice cognitive and measure in cv_predict data columns
            cv_predict=(cv_predict_df_unstack
                        .loc[subs,idx[measure,:,:,:,np_name]]
                        .values)
            #get list of analysis names
            #to get BootRollingStats objects from dict
            anals=[measure+'_'+np_name+'_'+x for x in anal_list]
            ##compute bootstrap measures
            bootstats=[
                ###simple correlations
                compute_corr(X=cv_predict,y=age),
                compute_corr(X=cv_predict,y=cognitive),
                ##gender only covar models
                compute_pcorr(covars=gender,X=cv_predict,y=age),
                compute_pcorr(covars=gender,X=cv_predict,y=cognitive),
                run_mediate(xvar_vec=age,yvar_vec=cognitive,
                            mediatevar_mat=cv_predict,
                            covar_vec=gender),
                ##gender + wb covar models
                compute_pcorr(covars=gender_wholebrain,X=cv_predict,y=age),
                compute_pcorr(covars=gender_wholebrain,X=cv_predict,y=cognitive),
                run_mediate(xvar_vec=age,yvar_vec=cognitive,
                            mediatevar_mat=cv_predict,
                            covar_vec=gender_wholebrain)
            ]
            #add boot stats to RollingBootStats objects
            for numanal in range(len(anal_list)):
                bootstats_dict[anals[numanal]].update(bootstats[numanal],num_boot)
                bootstats_dict[anals[numanal]].add_bootsample(bootstats[numanal],
                                                              num_boot)
    num_boot+=1 #add a boot sample to count

In [ ]:
#combine and format data
boot_stats_names=['empirical_mean','bootobj_mean','bootobj_sd','lower','upper']
boot_stats_list=[make_statdf(x) for x in boot_stats_names]
boot_stats_df=(pd.concat(boot_stats_list)
               .set_index(['stat','analysis'],append=True)
               [0]
              .sort_index(level=range(6))
              .unstack(['stat']))

In [ ]:
boot_stats_df.head()

In [ ]:
boot_stats_df.to_csv('../data/stats/cv_predict_models_5Fold.csv',na_rep='NA')

## Wholebrain analyses

In [12]:
##compute bootstrap measures
def run_parallel_bootstats(measure,np_name,anal):
    age=-npdata['Age'].values
    gender=npdata[['Gender']].values
    cognitive=npdata[np_name].values
    gender_wholebrain=npdata[
    ['Gender','_'.join(['wholebrain',measure])]].values
    wholebrain=wholebrain_dict[measure].loc[subs,:].values
    if anal=='mediate':
        stat_arr=run_mediate(xvar_vec=age,yvar_vec=cognitive,
                mediatevar_mat=wholebrain,
                covar_vec=gender)
    elif anal=='pcorr_age':
        stat_arr=compute_pcorr(covars=gender,X=wholebrain,y=age)
    elif anal=='pcorr_cog':
        stat_arr=compute_pcorr(covars=gender,X=wholebrain,y=cognitive)
    elif anal=='corr_cog':
        stat_arr=compute_corr(X=wholebrain,y=cognitive)
    else:
        raise ValueError('Invalid stat name')
    return (measure+'_'+np_name+'_'+anal,stat_arr)

In [13]:
#read in npdata and morphometry predictions obtained from cv_results
#and processed in make_cv_dfs.R script
npdatafilt=pd.read_csv('../data/np_filter_wb_gendernum.csv',
                       na_values='NA',index_col="subs")
wholebrain_dict={'thickness':(pd.read_pickle('/Volumes/Users/mbkranz/projects/ACT_Freesurfer_NewProc/'
                     'networks_ML/thickness_fwhm10_wholebrainfsaverage_df.pkl')
                              .reset_index(['measure'],drop=True)),
                 'area':(pd.read_pickle('/Volumes/Users/mbkranz/projects/ACT_Freesurfer_NewProc/'
                     'networks_ML/area_fwhm10_wholebrainfsaverage_df.pkl')
                              .reset_index(['measure'],drop=True))}

In [12]:
measure_list=['area','thickness']
np_name_list=['Memory','ExFunction']
hemilist=['rh','lh']
bootlist=['bootscore','bootmean']
#anal_list=['corr_cog','pcorr_age','pcorr_cog','mediate']
anal_list=['pcorr_age','pcorr_cog','mediate']
meas_key={'thickness':np.int(0),'area':np.int(1)}
hemi_key={'lh':np.int(0),'rh':np.int(1)}
numsamples=2000

In [17]:
import warnings
warnings.filterwarnings('ignore')

In [18]:
bootstats_dict={meas+'_'+np_name+'_'+anal:RollingBootStats(
        wholebrain_dict[meas].columns,numsamples)
                for meas in meas_key.iterkeys()
                for np_name in np_name_list 
                for anal in anal_list}
for num_boot in range(numsamples+1):
    ###resampled npdata and morph data
    subs,npdata=get_resampled_indices(npdatafilt,num_boot)
    #loop through each cognitive, morph, and stat measure
    #in parallel as it takes around 32 sec to run through 
    #the 3 stats6
    starttime=time.time()
    bootstats_list=Parallel(12)(delayed(run_parallel_bootstats)
                (measure,np_name,anal)
                for measure in meas_key.iterkeys()
                for np_name in np_name_list 
                for anal in anal_list)
    endtime=time.time()
    if num_boot%10==0:
        print('sample '+str(num_boot) + 
              ' stats took '+str(endtime-starttime))
    #add boot stats to RollingBootStats objects
    for anal_name,anal in bootstats_list:
        (bootstats_dict[anal_name]
         .update(anal,num_boot))

sample 0 stats took 35.1173670292
sample 10 stats took 29.2049238682
sample 20 stats took 29.2915360928
sample 30 stats took 29.7610719204
sample 40 stats took 29.0375299454
sample 50 stats took 28.7218580246
sample 60 stats took 32.8372189999
sample 70 stats took 29.2996580601
sample 80 stats took 29.0019950867
sample 90 stats took 29.2516129017
sample 100 stats took 29.226017952
sample 110 stats took 29.2366819382
sample 120 stats took 29.4712810516
sample 130 stats took 29.8404150009
sample 140 stats took 28.5745849609
sample 150 stats took 29.7118859291
sample 160 stats took 28.8551058769
sample 170 stats took 29.0228209496
sample 180 stats took 28.853012085
sample 190 stats took 28.644646883
sample 200 stats took 28.7501018047
sample 210 stats took 28.6975400448
sample 220 stats took 29.2881319523
sample 230 stats took 28.863008976
sample 240 stats took 28.7705690861
sample 250 stats took 28.7851390839
sample 260 stats took 28.2539288998
sample 270 stats took 29.0498218536
sample 

In [23]:
f=open('../data/wholebrain_bootstrap/bootstats_dict_4_17_18.pkl','wb')
pkl.dump(bootstats_dict,f)
f.close()

In [14]:
def getannot(annotname,hemi):
    annot_data=fs.read_annot(
        '/Applications/freesurfer/'
        'subjects/fsaverage/label/' + hemi + '.' + 
        annotname + '.annot')
    annot_hemi=pd.DataFrame({
            "annot_label" : annot_data[0],
            "vertex_index" : range(len(annot_data[0]))})
    return annot_hemi

In [17]:
def save_curvoverlay(analname,bsobj,hemi,bootmeas,save_as_pickle=True):
    hemi_index=hemi_key[hemi]
    annots=getannot('Yeo2011_7Networks_N1000',hemi)
    
    bs_df=pd.DataFrame({'empirical_mean':bsobj.empirical_mean,
                  'boot_mean':bsobj.bootobj_mean,
                  'boot_sd':bsobj.bootobj_sd},
                  index=bsobj.stat_labels)
    bs_df['boot_score']=bs_df['empirical_mean']/bs_df['boot_sd']
    bs_df_annot=(bs_df.loc[idx[:,hemi_key[hemi],:],:]
                .reset_index([0,'hemi'],drop=True)
                .join(annots,how='right'))
    curv_vals=(bs_df_annot.sort_index()
               [bootmeas]
               .values)
    if save_as_pickle:
        bs_df_annot.to_pickle('../data/wholebrain_bootstrap/'+hemi+'_'+analname+'_'+bootmeas+'.pkl')
    fs.write_morph_data(file_like=('../curvoverlays/'+
                                   hemi+'_'+analname+'_'+bootmeas+'.curv'),
                        values=curv_vals)

In [15]:
bootstats_dict=pkl.load(open('../data/wholebrain_bootstrap/bootstats_dict_4_17_18.pkl','rb'))

KeyboardInterrupt: 

In [18]:
for score in ['boot_mean','boot_score']:
    for anal_name,stats in bootstats_dict.iteritems():
        for hemi in hemi_key.keys():
            save_curvoverlay(anal_name,stats,hemi,score)